# The network object

https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.random.normal.html

https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.random.randn.html

https://stackoverflow.com/questions/21738383/python-difference-between-randn-and-normal

In [2]:
from keras.datasets import mnist
import numpy as np
(X_train_raw, y_train_raw), (X_test_raw, y_test_raw) = mnist.load_data()

X_train = np.reshape(X_train_raw, (60000, 784))
X_test = np.reshape(X_test_raw, (10000, 784))

def vectorize(j):
    """Return a 10-dimensional unit vector with a 1.0 in the jth
    position and zeroes elsewhere.  This is used to convert a digit
    (0...9) into a corresponding desired output from the neural
    network."""
    
    j = int(j)
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

y_train = np.array([vectorize(y) for y in y_train_raw])
y_test = np.array([vectorize(y) for y in y_test_raw])

In [8]:
import numpy as np

def sigmoid(z):
    return 1.0/1.0+np.exp(-z)
    

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))
    
    
class LayeredNetwork(object):    
    def __init__(self, layer_sizes):
        """
            Args:
                layer_sizes:
                    type: list of integers
                    description: list containing the sizes of each layer
                                 - first layer is the input layer
                                 - last layer is the output layer
                                 
            Returns:
                None
            Raises:
                Exception: "Please specify at least 2 layers" if len(layer_sizes) < 2
                
        """
        
        if len(layer_sizes) < 2:
            raise Exception("Please specify at least 2 layers")
        
        def get_random_biases(layer_sizes):
            biases = []
            for i in range(1, len(layer_sizes)):
                biases.append(np.random.randn(layer_sizes[i], 1))
                
            return biases
        
        def get_random_weights(layer_sizes):
            """
                Comment: weights[k] is a matrix of dimension layer_sizes[k] x layer_sizes[k-1]
                         - i,j th entry of this matrix corresponds to the forward edge from 
                         j th node in layer k-1 to i th node in layer k 
            """
            weights = []
            for i in range(1, len(layer_sizes)):
                weights.append(np.random.randn(layer_sizes[i], layer_sizes[i-1]))
                
            return weights    
               
            
                
        self.num_layers = len(layer_sizes)
        self.layer_sizes = layer_sizes
        
        """
            For every non-input layer, we associate a bias vector and a weight matrix.
            Note that len(biases) = len(weights) = len(layer_sizes) - 1
        """
        self.biases = get_random_biases(layer_sizes)
        self.weights = get_random_weights(layer_sizes)
        
    def show(self):
        print("The layer sizes are: {}".format(self.layer_sizes))
        print("The biases are:")
        for bias in self.biases:
            print(bias)
        print("The weights are:")
        for weight in self.weights:
            print(weight)
            
    def forward_pass(self, input_vector):
        x = input_vector
        
        
        
        for i in range(len(self.biases)):
            x =  sigmoid(np.dot(self.weights[i], x)  + self.biases[i])
    
    def evaluate(self, X, y):
        count = 0
        for i in range(len(X)):
            if get_label(X[i]) == y[i]:
                count = count + 1
        return count/len(X)   
     
    def SGD(self, X_train, y_train, num_epochs, mini_batch_size, learning_rate, X_val=None, y_val=None):
        
        
        def update_mini_batch(X, y, batch_indices, learning_rate):
            
            b_updates = np.zeros_like(self.biases)
            w_updates = np.zeros_like(self.weights)
            
            for i in range(len(batch_indices)):
                gradient_b, gradient_w = self.get_gradients(X[batch_indices[i]], y[batch_indices[i]])
                
                for i in range(len(self.biases)):
                    print("b[i] {}".format(b_updates.shape))
                    print("grad_b {}".format(gradient_b.shape))
                    b_updates[i] = b_updates[i] + learning_rate * gradient_b #/ len(mini_batch_size)
                    w_updates[i] = w_updates[i] + learning_rate * gradient_w #/ len(mini_batch_size)
                
            for i in range(self.biases):
                self.biases[i] = self.biases[i] + b_updates[i]
                self.weights[i] = self.weights[i] + w_updates[i]
            
    
        for i in range(num_epochs):
            row_indices = np.random.permutation(len(X_train))
            
            for i in range(0, len(X_train), mini_batch_size):
                batch_indices = row_indices[i: i+mini_batch_size]
                
                update_mini_batch(X_train, y_train, batch_indices, learning_rate)
                
                if not X_val:
                    self.evaluate(X_val, y_val)
                    
    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)                
                    
    def get_gradients(self, x, y):
        x = x.reshape(784, 1)
        y = y.reshape(10, 1)
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            print("w: {} a: {}".format(w.shape, activation.shape))
            z = np.dot(w, activation)+b
            print("z: {} dot {} b: {} b+b {}".format(z.shape, np.dot(w, activation).shape, b.shape, (b+b).shape))
            zs.append(z)
            activation = sigmoid(z)
            print("a: {}".format(activation.shape))
            activations.append(activation)
            
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return nabla_b, nabla_w
            
      
    

        
        
        
        
        
        

In [165]:
 """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

IndentationError: unexpected indent (<ipython-input-165-57e79cb0b4f4>, line 5)

In [9]:
net = LayeredNetwork([784, 15, 10])
training_data = [(x,y) for (x, y) in zip(X_train, y_train)]
test_data= [(x, y) for (x, y) in zip(X_test, y_test)]
net.SGD(X_train, y_train, 30, 10, 3.0, X_test, y_test)

w: (15, 784) a: (784, 1)
z: (15, 1) dot (15, 1) b: (15, 1) b+b (15, 1)
a: (15, 1)
w: (10, 15) a: (15, 1)
z: (10, 1) dot (10, 1) b: (10, 1) b+b (10, 1)
a: (10, 1)
b[i] (2,)


AttributeError: 'list' object has no attribute 'shape'

In [82]:
network = LayeredNetwork([10,5,1])
network.show()

The layer sizes are: [10, 5, 1]
The biases are:
[[-0.62129554]
 [-0.55199231]
 [ 1.2527846 ]
 [ 2.41966621]
 [-0.13425269]]
[[-0.70487286]]
The weights are:
[[ 0.30627786  1.25222505  1.64558533  0.43583958 -1.29095652  0.09195977
   0.23545705 -0.99088548 -0.41734031  0.6942187 ]
 [-0.26196643  0.62520059 -0.5942989   0.91126444  0.32896317  0.28578463
  -0.41423441  0.98416404 -1.2821212   0.46423828]
 [-0.10355521  0.76920646  0.15448216  0.57031335 -0.18087896 -1.11408102
   2.15013875  1.05866105 -0.63214082  0.45344996]
 [ 1.09282056 -1.00592598  0.46920034 -0.2927609  -1.07724381 -1.36091972
   0.72087421 -0.05823585 -0.76454616  0.70834811]
 [-0.04185681  0.72927593 -0.78629115 -0.37286283 -0.86458217 -0.37711559
  -0.66554224 -0.00848353  0.65966804 -0.24495533]]
[[ 0.5052734  -0.37977215  0.7024904   0.67526387  0.48284119]]


# SCRATCH CODE SPACE BELOW SCRATCH CODE SPACE BELOW

In [3]:
import numpy as np

class Network(object):
    """
        sizes is a list of integers indicating the number of neurons in each layer
        Network([100, 50, 1]) initializes a neural network with 
            - first layer containing 100 neurons is the input layer
            - second layer containing 50 neurons is the hidden layer
            - final layer containing 1 neuron is the output layer
             
    """
        
    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) 
                        for x, y in zip(sizes[:-1], sizes[1:])]

In [4]:
np.random.randn(5,1)

array([[-0.9630888 ],
       [-0.57947231],
       [ 0.26961107],
       [ 0.16915313],
       [ 1.25001683]])

In [5]:
np.random.randn(2,3)

array([[ 0.49233341,  0.27856956, -0.84535872],
       [-1.37835673, -0.25390091, -0.36282913]])

In [6]:
def sigmoid(z):
    """
        When the input z is a vector or Numpy array, 
        Numpy automatically applies the function sigmoid elementwise, 
        that is, in vectorized form.
    """
    return 1.0/(1.0+np.exp(-z))

In [7]:
 def feedforward(self, a):
        """Return the output of the network if "a" is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

In [ ]:
def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

    
def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The "mini_batch" is a list of tuples "(x, y)", and "eta"
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw 
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb 
                       for b, nb in zip(self.biases, nabla_b)


def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The "training_data" is a list of tuples
        "(x, y)" representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If "test_data" is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        if test_data: n_test = len(test_data)
            
        n = len(training_data)
        for j in xrange(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in xrange(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
                
            """
            """    
            if test_data:
                print "Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data), n_test)
            else:
                print "Epoch {0} complete".format(j)

In [23]:
"""
network.py
~~~~~~~~~~

A module to implement the stochastic gradient descent learning
algorithm for a feedforward neural network.  Gradients are calculated
using backpropagation.  Note that I have focused on making the code
simple, easily readable, and easily modifiable.  It is not optimized,
and omits many desirable features.
"""

#### Libraries
# Standard library
import random

# Third-party libraries
import numpy as np

class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data), n_test))
            else:
                print("Epoch {0} complete".format(j))
                

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

In [83]:
from keras.datasets import mnist

(X_train_raw, y_train_raw), (X_test_raw, y_test_raw) = mnist.load_data()

In [84]:
from keras.datasets import mnist

(X_train_raw, y_train_raw), (X_test_raw, y_test_raw) = mnist.load_data()

X_train = np.reshape(X_train_raw, (60000, 784))
X_test = np.reshape(X_test_raw, (10000, 784))

def vectorize(j):
    """Return a 10-dimensional unit vector with a 1.0 in the jth
    position and zeroes elsewhere.  This is used to convert a digit
    (0...9) into a corresponding desired output from the neural
    network."""
    
    j = int(j)
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

y_train = np.array([vectorize(y) for y in y_train_raw])
y_test = np.array([vectorize(y) for y in y_test_raw])




TypeError: 'list' object cannot be interpreted as an integer

In [12]:
"""
mnist_loader
~~~~~~~~~~~~

A library to load the MNIST image data.  For details of the data
structures that are returned, see the doc strings for ``load_data``
and ``load_data_wrapper``.  In practice, ``load_data_wrapper`` is the
function usually called by our neural network code.
"""

#### Libraries
# Standard library
import cPickle
import gzip

# Third-party libraries
import numpy as np

def load_data():
    """Return the MNIST data as a tuple containing the training data,
    the validation data, and the test data.

    The ``training_data`` is returned as a tuple with two entries.
    The first entry contains the actual training images.  This is a
    numpy ndarray with 50,000 entries.  Each entry is, in turn, a
    numpy ndarray with 784 values, representing the 28 * 28 = 784
    pixels in a single MNIST image.

    The second entry in the ``training_data`` tuple is a numpy ndarray
    containing 50,000 entries.  Those entries are just the digit
    values (0...9) for the corresponding images contained in the first
    entry of the tuple.

    The ``validation_data`` and ``test_data`` are similar, except
    each contains only 10,000 images.

    This is a nice data format, but for use in neural networks it's
    helpful to modify the format of the ``training_data`` a little.
    That's done in the wrapper function ``load_data_wrapper()``, see
    below.
    """
    f = gzip.open('../data/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = cPickle.load(f)
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    """Return a tuple containing ``(training_data, validation_data,
    test_data)``. Based on ``load_data``, but the format is more
    convenient for use in our implementation of neural networks.

    In particular, ``training_data`` is a list containing 50,000
    2-tuples ``(x, y)``.  ``x`` is a 784-dimensional numpy.ndarray
    containing the input image.  ``y`` is a 10-dimensional
    numpy.ndarray representing the unit vector corresponding to the
    correct digit for ``x``.

    ``validation_data`` and ``test_data`` are lists containing 10,000
    2-tuples ``(x, y)``.  In each case, ``x`` is a 784-dimensional
    numpy.ndarry containing the input image, and ``y`` is the
    corresponding classification, i.e., the digit values (integers)
    corresponding to ``x``.

    Obviously, this means we're using slightly different formats for
    the training data and the validation / test data.  These formats
    turn out to be the most convenient for use in our neural network
    code."""
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    """Return a 10-dimensional unit vector with a 1.0 in the jth
    position and zeroes elsewhere.  This is used to convert a digit
    (0...9) into a corresponding desired output from the neural
    network."""
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

ModuleNotFoundError: No module named 'cPickle'

In [11]:
import mnist_loader
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()

ModuleNotFoundError: No module named 'mnist_loader'